In [19]:
import carla 
import math 
import random 
import time 
import numpy as np
import cv2

np.bool = np.bool_ #per retrocompatibilità

# connessione al simulatore, recupero blueprints e spawn points
client = carla.Client('localhost', 2000) 
world = client.get_world()
bp_lib = world.get_blueprint_library()  
spawn_points = world.get_map().get_spawn_points() 

In [20]:
# aggiunge veicolo
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[79])

# pov spettatore dietro veicolo
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

# aggiunge 50 macchine scelte casualmente dal blueprint
for i in range(50): 
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points)) #trova spawn liberi


In [21]:
# setta l'autopilota per tutte le macchine istanziate
for v in world.get_actors().filter('*vehicle*'): 
    v.set_autopilot(True) 

In [22]:
# posizione iniziale per tutte le telecamere
camera_init_trans = carla.Transform(carla.Location(z=2))

# aggiunge 6 tipi di *camera*
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation') 
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle)

inst_camera_bp = bp_lib.find('sensor.camera.instance_segmentation') 
inst_camera = world.spawn_actor(inst_camera_bp, camera_init_trans, attach_to=vehicle)

depth_camera_bp = bp_lib.find('sensor.camera.depth') 
depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=vehicle)

dvs_camera_bp = bp_lib.find('sensor.camera.dvs') 
dvs_camera = world.spawn_actor(dvs_camera_bp, camera_init_trans, attach_to=vehicle)

opt_camera_bp = bp_lib.find('sensor.camera.optical_flow') 
opt_camera = world.spawn_actor(opt_camera_bp, camera_init_trans, attach_to=vehicle)

In [23]:
# callbacks per ciascuna telecamera che ritorna l'immagine nella mappa data_dict[]
def rgb_callback(image, data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    
def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette) #conversione dei colori per miglior contrasto
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def inst_callback(image, data_dict):
    data_dict['inst_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth) #conversione dei colori per miglior contrasto
    data_dict['depth_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    
def opt_callback(data, data_dict):
    image = data.get_color_coded_flow()
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    img[:,:,3] = 255
    data_dict['opt_image'] = img
    
def dvs_callback(data, data_dict): #dynamic vision camera, per ogni cambiamento di intensità nell'immagine ritorna un evento
    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([ #cattura le coordinate degli eventi
                ('x', np.uint16), ('y', np.uint16), ('t', np.int64), ('pol', np.bool)])) 
    data_dict['dvs_image'] = np.zeros((data.height, data.width, 4), dtype=np.uint8)
    dvs_img = np.zeros((data.height, data.width, 3), dtype=np.uint8)
    dvs_img[dvs_events[:]['y'], dvs_events[:]['x'], dvs_events[:]['pol'] * 2] = 255 #colora i pixel degli eventi e li mostra nell'immagine
    data_dict['dvs_image'][:,:,0:3] = dvs_img
    

In [24]:
# prende le dimensioni di default di ogni telecamera, sono uguali per tutte quindi usa camera_bp
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

#inizializza il dizionario delle immagini per ciascun sensore, sarà aggiornato dalle funziona di callback
sensor_data = {'rgb_image': np.zeros((image_h, image_w, 4)),
               'sem_image': np.zeros((image_h, image_w, 4)),
               'depth_image': np.zeros((image_h, image_w, 4)),
               'dvs_image': np.zeros((image_h, image_w, 4)),
               'opt_image': np.zeros((image_h, image_w, 4)), 
               'inst_image': np.zeros((image_h, image_w, 4))}

# finestra open cv per tutte le telecamere. E' divisa in quadranti, uno per ciascun tipo
cv2.namedWindow('All cameras', cv2.WINDOW_AUTOSIZE)

# disposizione quadranti nella finestra opencv
top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image'], sensor_data['inst_image']), axis=1)
lower_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image'], sensor_data['opt_image']), axis=1)
tiled = np.concatenate((top_row, lower_row), axis=0)


cv2.imshow('All cameras',tiled)
cv2.waitKey(1)

# sensori in ascolto
camera.listen(lambda image: rgb_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))
inst_camera.listen(lambda image: inst_callback(image, sensor_data))
depth_camera.listen(lambda image: depth_callback(image, sensor_data))
dvs_camera.listen(lambda image: dvs_callback(image, sensor_data))
opt_camera.listen(lambda image: opt_callback(image, sensor_data))

# Indefinite while loop
while True:
    
    # disposizione quadranti nella finestra opencv
    top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image'], sensor_data['inst_image']), axis=1)
    lower_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image'], sensor_data['opt_image']), axis=1)
    tiled = np.concatenate((top_row, lower_row), axis=0)
       
    # finestra a video
    cv2.imshow('All cameras',tiled)
    
    # interrompi se l'utente preme 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# ferma i sensori, chiude la finestra a video e elimina i veicoli dal simulatore
camera.stop()
sem_camera.stop()
inst_camera.stop()
depth_camera.stop()
dvs_camera.stop()
opt_camera.stop()
cv2.destroyAllWindows()
for v in world.get_actors().filter('*vehicle*'): 
    v.destroy()